In [1]:
import os
import numpy as np
import pandas as pd
import pickle as pkl
import plotly.express as px
import plotly.graph_objects as go

from scipy.spatial.transform import Rotation as R
from keypoint_util import KeypointSelector

def attach_grip_states(df, grip):
    new_df = df.copy()
    new_df['action'] = None
    for i, row in grip.iterrows():
        check_time = row['Timestamp']
        new_df.iloc[(df['Time'] - check_time).abs().argsort()[:1], -1] = row['Gripper state']
    return new_df

folder = os.path.join('ndi_servo', '2022-04-13-morning')
instances = sorted(os.listdir(folder))
all_dfs = []
for instance in instances:
    instance_folder = os.path.join(folder, instance)
    file_set = sorted(os.listdir(instance_folder))[:2]
    file1, file2 = file_set
    # with open(os.path.join(data_folder, data497, 'labeled_actions_new.pkl'), 'rb') as f:
    #     label_actions = pkl.load(f)
    # trajectory_file = os.path.join(data_folder, data497, 'obj_pose_trajectory_interpolated_median.h5')
    # trajectory_dataset = pd.read_hdf(trajectory_file)
    grip_file = os.path.join(instance_folder, file1)
    state_file = os.path.join(instance_folder, file2)
    grip_states = pd.read_csv(grip_file)
    pos_states = pd.read_csv(state_file, index_col='Unnamed: 0')
    dframe = attach_grip_states(pos_states, grip_states)
    dframe.loc[:,['Rx', 'Ry', 'Rz']]
    all_dfs.append(dframe)
instances

['260650',
 '816303',
 '838539',
 '838540',
 '838542',
 '838544',
 '838545',
 '838546',
 '838547']

In [2]:
# For drawing axis
for df in all_dfs:
    xax = [1,0,0]
#     yax = [0,1,0]
#     zax = [0,0,1]
    r = R.from_rotvec(df.loc[:,['Rx', 'Ry', 'Rz']])
    df.loc[:, ['xu','xv','xw']] = r.apply(xax)
#     df.loc[:, ['yu','yv','yw']] = r.apply(yax)
#     df.loc[:, ['zu','zv','zw']] = r.apply(zax)

# Setup key point search parameters
selector = KeypointSelector(16, 8, 0.07, 0.05, rotation_max=.7, 
                            velocity_window_size=5, rotation_window_size=5)

In [9]:
# for f, df in enumerate(all_dfs):
#     cond_df, keypoints, vel_df, rot_df = selector.search(all_dfs[f])
#     grasp_indices = cond_df.index[cond_df['condition'] == 'grasp'].tolist()
#     task_periods = [grasp_indices[i*2:i*2+2] for i in range(int(len(grasp_indices)/2))]

    # Set start and end frames
#     start, end = task_periods[0]
#     save_df = cond_df.iloc[start:end]
#     save_df.to_csv("df_{}.csv".format(instances[f]))

C:\Users\YC\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
# summarize the key points from all demonstration
tasks_keypoints = {i:[] for i in range(6)}
for j in range(len(instances)):
    cond_df, keypoints, vel_df, rot_df = selector.search(all_dfs[j])
    grasp_indices = cond_df.index[cond_df['condition'] == 'grasp'].tolist()
    task_periods = [grasp_indices[i*2:i*2+2] for i in range(int(len(grasp_indices)/2))]
    keypoint_df = cond_df.copy().iloc[keypoints]
    for k, period in enumerate(task_periods):
        start, end = period
        keypoint_sliced_df = keypoint_df.loc[[i for i in keypoint_df.index if i in range(start, end+1)]]
        key_cond = keypoint_sliced_df.condition
        key_cap = [key[0].upper() for key in key_cond[key_cond!='grasp'].tolist()]
        tasks_keypoints[k].append(key_cap)

In [ ]:
tasks_keypoints[1]

In [ ]:
# 0: teabag to cup
# 1: pitcher to tap
# 2: turn tap
# 3: wait
# 4: turn tap back
# 5: pour pitcher to cup
# List all close and open grasp frame indices.
index = 1
cond_df, keypoints, vel_df, rot_df = selector.search(all_dfs[index])
grasp_indices = cond_df.index[cond_df['condition'] == 'grasp'].tolist()
task_periods = [grasp_indices[i*2:i*2+2] for i in range(int(len(grasp_indices)/2))]

# Set start and end frames
start, end = task_periods[5]

traj_df = cond_df.copy().iloc[start:end+1]
keypoint_df = cond_df.copy().iloc[keypoints]
keypoint_sliced_df = keypoint_df.loc[[i for i in keypoint_df.index if i in range(start, end+1)]]
fig_line = px.line_3d(traj_df, x='x', y='y', z='z')
fig_pt = px.scatter_3d(keypoint_sliced_df, x='x', y='y', z='z', color='Time', symbol='condition')
# fig3 = go.Figure(data=go.Cone(x=keypoint_sliced_df['x'], y=keypoint_sliced_df['y'], z=keypoint_sliced_df['z'], 
#                              u=keypoint_sliced_df['u'], v=keypoint_sliced_df['v'], w=keypoint_sliced_df['w'],
#                              sizemode="absolute", sizeref=2, anchor="tip"))
fig_main= go.Figure(data=fig_line.data + fig_pt.data)
fig_main.update_traces(marker=dict(size=5))
# fig4.update_layout(
#         scene = dict(
#             # aspectratio=dict(x=1, y=1, z=1), # <---- tried this too
#             aspectmode='cube'
#         ))
fig_main.show()

vector_df = keypoint_sliced_df
figx = go.Figure(data=go.Cone(x=vector_df['x'], y=vector_df['y'], z=vector_df['z'], 
                             u=vector_df['xw'], v=vector_df['xv'], w=vector_df['xu']))
# figy = go.Figure(data=go.Cone(x=vector_df['x'], y=vector_df['y'], z=vector_df['z'], 
#                              u=vector_df['yu'], v=vector_df['yv'], w=vector_df['yw']))
# figz = go.Figure(data=go.Cone(x=vector_df['x'], y=vector_df['y'], z=vector_df['z'], 
#                              u=vector_df['zu'], v=vector_df['zv'], w=vector_df['zw']))
fig_vec = go.Figure(data=figx.data + fig_line.data)

# fig.update_layout(scene=dict(aspectratio=dict(x=1, y=1, z=1)))
fig_vec.show()

In [ ]:
m = 1184
vel_df.iloc[m-2:m+3]

In [ ]:
rot_df.iloc[start:end+1][keypoint_sliced_df.index]

In [ ]:
vel_df.iloc[start:end+1][keypoint_sliced_df.index]

In [ ]:
keypoint_sliced_df

In [ ]:
# relevant_indices = []
# for period in task_periods:
#     relevant_indices += range(*period)

In [ ]:
# relevant_df = cond_df.iloc[relevant_indices]
# cut_df = keypoint_df
# fig_all_line = px.line_3d(relevant_df, x='x', y='y', z='z')
# fig_all_x = go.Figure(data=go.Cone(x=cut_df['x'], y=cut_df['y'], z=cut_df['z'], 
#                              u=cut_df['xu'], v=cut_df['xv'], w=cut_df['xw']))
# # figy = go.Figure(data=go.Cone(x=vector_df['x'], y=vector_df['y'], z=vector_df['z'], 
# #                              u=vector_df['yu'], v=vector_df['yv'], w=vector_df['yw']))
# # figz = go.Figure(data=go.Cone(x=vector_df['x'], y=vector_df['y'], z=vector_df['z'], 
# #                              u=vector_df['zu'], v=vector_df['zv'], w=vector_df['zw']))
# fig_all = go.Figure(data=fig_all_x.data+fig_all_line.data)
# # fig_all.update_traces(marker=dict(size=5))
# fig_all.show()

In [ ]:
target = 313
print('Smoothed velocity:\n', velo_df.iloc[target-1:target+20])
print('Smoothed rotation:\n', rot_df.iloc[target-1:target+2])
print(cond_df.iloc[target-1:target+2])